<a href="https://colab.research.google.com/github/gacemiamine77/gacemiamine77/blob/main/LST_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.1 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=32c28b4fb2d4d4628b9a602b3cbec94424b72d0c14a05173f94f1b

In [2]:
!pip install earthpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 56.2 MB/s eta 0:00:00


In [3]:
import geemap
import ee
from google.colab import drive
import folium

In [4]:
drive.mount('/content/drive')
ee.Authenticate()
ee.Initialize()

Mounted at /content/drive
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=okIzaFdtqfPHQlD5Zg5_s-NMGF7caB6bjBLofGUAJ7E&tc=kY5pjpQRFl-281m-G3kPRpkD7IR4j_rA302pGazG-j4&cc=NmNdJncnPXEgJuKyN73yu6LlOb67-3erFwti8iSO5fU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VMaK_pNN4AD3SXCWfoOvAMO7De7dXWak9DqM_1d4ijjZ-zRiSlg224

Successfully saved authorization token.


In [5]:
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME',"Sidi Bel Abbes")).geometry()

In [108]:
def getLST(image):
  
    #B=image.select('B10').rename("LST1")
   #B11=image.select('B11')
   # Tb=image.select('B10'),
   # emissivity=image.select('B11')*0.01
   # radiance = image.expression('(0.0003342 * B10) + 0.1') #radiance
   
    lst = image.expression('Tb+(1+(0.00115*(Tb/1.438)*log(emissivity))-273.15)', {
        'Tb': image.select('B10'),
        'emissivity': image.select('B11')
         })
    #lst = radiance.expression('(Tb / (1 + (0.00115 * (Tb / 1.438)) * log(emissivity))) - 273.15') #lst en degree c
    #lst = lst.rename('LST')"""
    lst = lst.rename("HHH")
    image = image.addBands(lst)
    
    return (image)


EEException: ignored

In [127]:
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
            .filterBounds(aoi)\
            .filterDate('2021-05-01', '2021-05-30').map(getLST).median()


In [131]:
B1=landsat.select('B1')
radiance = landsat.expression('(0.0003342 * B1) + 0.1') #radiance
print(radiance.getInfo())

EEException: ignored

In [110]:
LST1=getLST(landsat)
b=LST1.select("HHH")


In [116]:
color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":60, 'palette':color}

In [117]:
vizParams = {
  'bands': ['HHH'],
  'min': 0,
  'max': 0.5
  
}

In [118]:
map1 = geemap.Map()
map1.centerObject(aoi, 8)
map1.addLayer(b.clip(aoi),pallete, "HHH")

map1.addLayerControl()
map1


Map(center=[34.70841918557641, -0.5073845667977872], controls=(WidgetControl(options=['position', 'transparent…